In [1]:
from tqdm.auto import tqdm
import math
from decimal import Decimal

In [2]:
# Define the PRIME_SET and the Prime function
PRIME_SET = set([2])

In [3]:
RHO_MAP = {}

In [4]:
def break_down(n: int, base: int) -> tuple[int, list[tuple[int, int]]]:
  exponent = 0
  while n % base == 0:
    n = n // base
    exponent += 1
  if exponent > 0:
    return n, [(base, exponent)]
  return n, []

In [5]:
def is_prime(k: int, add_to_PRIME_SET: bool = False):
  for prime in PRIME_SET:
    if k % prime == 0:
      return False
  if add_to_PRIME_SET:
    PRIME_SET.add(k)
  return True

In [6]:
# Define the Rho function
def rho_prob_base(n: int, base = 2,
                  inc = 1,
                  threshold = 10**6,
                  enable_tqdm = False, trace = False) -> \
                  tuple[list[tuple[int, int]], float]:
  pairs: list[tuple[int, int]] = RHO_MAP.get(n, [])
  number = n
  range_bar = tqdm(
      total=n,
      initial=base,
      disable=not enable_tqdm,
      unit_scale=True,
  )
  root = Decimal(n).sqrt()

  def calc_prob():
    return min(1.00, (threshold / pairs[-1][0])) if pairs else 1.00

  if pairs:
      return pairs, calc_prob()

  while base < n:
    if base in PRIME_SET or is_prime(base, add_to_PRIME_SET=True):
      if number == 1:
        range_bar.update(n - base)
        range_bar.close()
        return pairs, calc_prob()
      if base > root or base > threshold:
        pairs.append((number, 1))
        range_bar.update(n - base)
        range_bar.close()
        return pairs, calc_prob()
      if number < n:
        rho_bases, prob = rho_prob_base(number,
                                        base,
                                        inc,
                                        threshold,
                                        enable_tqdm and trace,
                                        trace)
        pairs.extend(rho_bases)
        range_bar.update(n - base)
        range_bar.close()
        return pairs, calc_prob()
      number, pair = break_down(number, base)
      pairs.extend(pair)
    base += inc
    range_bar.update(inc)
  if number > 1:
    pairs.append((number, 1))
  range_bar.close()
  return pairs, calc_prob()

In [8]:
for num in tqdm(range(2, 1_000_002), desc="Rho iterations"):
  rho_nums, _ = rho_prob_base(num, enable_tqdm=False, trace=False)
  RHO_MAP[num] = rho_nums

Rho iterations:   0%|          | 0/1000000 [00:00<?, ?it/s]